<a href="https://colab.research.google.com/github/heewonLEE2/Data-Ai-Colab/blob/main/CV/YOLO_v1_%EA%B5%AC%ED%98%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ✅ **설계 목표**
논문 그대로 구현하려면 코드가 너무 길어지니, PyTorch 스타일로 깔끔하게 모듈화하겠습니다.
1. 입력: (Batch_Size, 3, 448, 448) (논문은 448x448 해상도를 씁니다)
2. 출력: (Batch_Size, 7, 7, 30) (PASCAL VOC 기준: $S=7, B=2, C=20$)
3. 구성:
- CNNBlock: 컨볼루션 + 배치정규화(옵션) + LeakyReLU
- Yolov1 Class: 전체 네트워크 조립 + 마지막 FC 레이어 + Reshape

## **1. 기본 블록 만들기 (CNNBlock)**

논문에서는 Conv -> LeakyReLU를 반복합니다. (현대적인 구현에서는 학습 안정을 위해 BatchNorm을 추가)

In [1]:
import torch
import torch.nn as nn
# YOLO v1 논문의 핵심 아키텍처 정보
# 튜플 구조: (kernel_size, filters, stride, padding)
# "M"은 MaxPool (kernel=2, stride=2)을 의미
architecture_config = [
    (7, 64, 2, 3),
    "M",
    (3, 192, 1, 1),
    "M",
    (1, 128, 1, 0),
    (3, 256, 1, 1),
    (1, 256, 1, 0),
    (3, 512, 1, 1),
    "M",
    # 리스트 구조: [(커널, 필터, 스트라이드, 패딩), 반복횟수]
    [(1, 256, 1, 0), (3, 512, 1, 1), 4],
    (1, 512, 1, 0),
    (3, 1024, 1, 1),
    "M",
    [(1, 512, 1, 0), (3, 1024, 1, 1), 2],
    (3, 1024, 1, 1),
    (3, 1024, 2, 1),
    (3, 1024, 1, 1),
    (3, 1024, 1, 1),
]

class CNNBlock(nn.Module):
    def __init__(self, in_channels, out_channels, **kwargs):
        super(CNNBlock, self).__init__()
        # bias=False를 쓰는 이유: BatchNorm을 쓰면 bias가 의미가 없어짐 (상쇄됨)
        self.conv = nn.Conv2d(in_channels, out_channels, bias=False, **kwargs)
        self.batchnorm = nn.BatchNorm2d(out_channels) # 논문엔 없지만 학습 안정성을 위해 추가
        self.leakyrelu = nn.LeakyReLU(0.1) # 논문에서 slope=0.1 명시

    def forward(self, x):
        return self.leakyrelu(self.batchnorm(self.conv(x)))

# **2. 전체 모델 조립하기 (Yolov1)**

여기서 눈여겨볼 점은 _create_conv_layers 메소드와 forward의 마지막 reshape입니다.

In [6]:
class Yolov1(nn.Module):
    def __init__(self, in_channels=3, **kwargs):
        super(Yolov1, self).__init__()
        self.architecture = architecture_config
        self.in_channels = in_channels

        # 1. 컨볼루션 레이어 생성 (Darknet 부분)
        self.darknet = self._create_conv_layers(self.architecture)

        # 2. 완전 연결 레이어 (Fully Connected Layers) - 여기가 'Head' 부분
        self.fcs = self._create_fcs(**kwargs)

    def forward(self, x):
        x = self.darknet(x)
        # Flatten: FC 레이어에 넣기 위해 1차원으로 폅니다.
        x = torch.flatten(x, start_dim=1)
        x = self.fcs(x)
        return x

    def _create_conv_layers(self, architecture):
        layers = []
        in_channels = self.in_channels

        for x in architecture:
            if type(x) == tuple: # (7, 64, 2, 3) 같은 일반 Conv 층
                layers += [
                    CNNBlock(
                        in_channels, x[1], kernel_size=x[0], stride=x[2], padding=x[3]
                    )
                ]
                in_channels = x[1] # 다음 레이어의 입력 채널 업데이트

            elif type(x) == str: # "M" Maxpool
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]

            elif type(x) == list: # 반복되는 구조 [(...)..., 반복횟수]
                conv1 = x[0]
                conv2 = x[1]
                num_repeats = x[2]

                for _ in range(num_repeats):
                    layers += [
                        CNNBlock(
                            in_channels, conv1[1], kernel_size=conv1[0], stride=conv1[2], padding=conv1[3]
                        )
                    ]
                    layers += [
                        CNNBlock(
                            conv1[1], conv2[1], kernel_size=conv2[0], stride=conv2[2], padding=conv2[3]
                        )
                    ]
                    in_channels = conv2[1]

        return nn.Sequential(*layers)

    def _create_fcs(self, split_size=7, num_boxes=2, num_classes=20):
        S, B, C = split_size, num_boxes, num_classes
        return nn.Sequential(
            nn.Flatten(),
            # 논문에 따르면: 448x448 입력 -> 최종 Conv 출력은 7x7x1024
            nn.Linear(1024 * S * S, 4096),
            nn.LeakyReLU(0.1),
            nn.Dropout(0.5), # 과적합 방지
            # 마지막 출력: S*S*(C + B*5) -> 7*7*30 = 1470
            nn.Linear(4096, S * S * (C + B * 5)),
        )

## **3. 동작 테스트 (검증)**

In [7]:
def test():
    # PASCAL VOC 기준 설정 (S=7, B=2, C=20)
    split_size = 7
    num_boxes = 2
    num_classes = 20

    # 모델 인스턴스 생성
    model = Yolov1(split_size=split_size, num_boxes=num_boxes, num_classes=num_classes)

    # 더미 데이터 생성 (Batch Size=2, 3채널, 448x448 이미지)
    x = torch.randn((2, 3, 448, 448))

    # 순전파 (Forward Pass)
    print("모델 실행 중...")
    output = model(x)

    # 텐서 형태 확인
    # 우리가 원하는 형태: (Batch, 1470) -> 나중에 Loss 계산 시 (Batch, 7, 7, 30)으로 변환해서 씀
    # 혹은 모델 forward 안에서 reshape을 해줘도 됨.
    print(f"FC Output Shape: {output.shape}")

    # 최종적으로 우리가 다루기 편한 형태로 Reshape
    final_output = output.reshape(-1, split_size, split_size, (num_classes + num_boxes * 5))
    print(f"Final Reshaped Shape: {final_output.shape}")

In [8]:
test()

모델 실행 중...
FC Output Shape: torch.Size([2, 1470])
Final Reshaped Shape: torch.Size([2, 7, 7, 30])


## **Code Review**

### 1. config 리스트의 중요성:

- 논문처럼 깊은 네트워크를 짤 때 nn.Conv2d를 24번 하드코딩하면 유지보수가 불가능합니다.

- 저런 설정 리스트(architecture_config)를 만들어두고 파싱하는 방식은 YOLO 공식 구현체(Darknet C코드)나 나중에 나올 YOLO v3 PyTorch 구현에서도 계속 쓰이는 표준 패턴입니다.

### 2. 4096 노드의 압박:
- nn.Linear(1024 * 7 * 7, 4096) 부분을 보세요. 입력 뉴런이 약 5만 개, 출력이 4천 개입니다.

- 여기서만 파라미터가 약 2억 개(200MB) 가 생성됩니다.

- YOLO v1이 모델 파일 크기가 컸던 이유가 바로 이 FC Layer(헤드) 때문입니다. (나중 버전인 v2부터는 이걸 없애고 완전한 FCN 구조로 갑니다.)

### 3. 마지막 reshape:
- nn.Linear는 1차원 벡터를 뱉습니다 (1470개).

- 하지만 우리는 이걸 $7 \times 7$ 그리드로 해석해야 하죠.

- 그래서 코드 밖에서나 안에서 반드시 (7, 7, 30)으로 다시 묶어주는 과정이 필요합니다.

## **4. 손실 함수(Loss Function) 구현**

### 1. Loss Function의 3가지 핵심 목표
YOLO Loss는 단순히 "틀린 만큼 벌점"을 주는 게 아니라, 상황에 따라 벌점을 다르게 줍니다.
- 좌표(Box) 에러: "객체가 있는 곳"이면 좌표를 아주 정교하게 맞춰야 해! ($\lambda_{coord} = 5$)
- No-Object 에러: "배경(빈 공간)"이면 박스가 없다고 확실하게 말해! ($\lambda_{noobj} = 0.5$)
- 크기 보정: 작은 물체의 1cm 오차가 큰 물체의 1cm 오차보다 훨씬 크다! ($\sqrt{w}, \sqrt{h}$ 사용)

### 2. 사전 준비: IOU 계산 함수 (Utils)
Loss를 계산하려면, 모델이 예측한 두 개의 박스(Box1, Box2) 중 어느 것이 정답(Ground Truth)과 더 비슷한지(Responsible) 알아내야 합니다. 이를 위해 intersection_over_union 함수가 필요합니다.

(이 함수는 별도 파일 utils.py에 있다고 가정하고 가져다 쓰지만, 여기서는 이해를 위해 핵심 로직만 간단히 포함하겠습니다.)

In [9]:
# IOU 계산 함수 (Loss 안에서 쓰임)
def intersection_over_union(boxes_preds, boxes_labels, box_format="midpoint"):
    """
    boxes_preds: (Batch, 7, 7, 4) - 예측한 박스 좌표
    boxes_labels: (Batch, 7, 7, 4) - 정답 박스 좌표
    반환값: (Batch, 7, 7, 1) - IOU 점수
    """
    # ... (상세 구현은 생략하고 로직만 설명) ...
    # 1. 박스의 좌표(x1,y1,x2,y2)를 구합니다.
    # 2. 교집합(Intersection) 영역의 넓이를 구합니다.
    # 3. 합집합(Union) 영역의 넓이를 구합니다.
    # 4. Intersection / Union 을 반환합니다.

    # 편의상 코드가 길어지니 일단 Pytorch 내장 함수나 약식으로 대체하지 않고
    # 실제 구현시엔 꼭 정석대로 구현된 함수를 써야 합니다.
    # 지금은 개념 이해를 위해 'iou' 변수가 있다고 가정하고 넘어갑니다.
    pass

### 3. YoloLoss 구현 (메인 코드)

데이터 구조 가정 (Output Tensor: 30채널):

- 0~19: 클래스 확률 (20개)

- 20: 신뢰도 1 (Confidence Score 1)

- 21~24: 박스 1 (x, y, w, h)

- 25: 신뢰도 2 (Confidence Score 2)

- 26~29: 박스 2 (x, y, w, h)

In [10]:
class YoloLoss(nn.Module):
    def __init__(self, S=7, B=2, C=20):
        super(YoloLoss, self).__init__()
        self.mse = nn.MSELoss(reduction="sum") # "sum"을 씁니다 (논문 수식 따름)
        self.S = S
        self.B = B
        self.C = C
        self.lambda_noobj = 0.5
        self.lambda_coord = 5

    def forward(self, predictions, target):
        # predictions shape: (Batch, 7*7*30) -> (Batch, 7, 7, 30)으로 변경
        predictions = predictions.reshape(-1, self.S, self.S, self.C + self.B * 5)

        # ---------------------------------------------------------
        # 1. 어떤 박스가 책임(Responsible)을 질지 결정하기 (IOU 계산)
        # ---------------------------------------------------------
        # 정답 박스 좌표 (x,y,w,h) 가져오기 (Target은 21~24에 좌표가 있다고 가정)
        # Target 구조: [class(20), conf(1), x,y,w,h(4)] = 총 25개 채널이라 가정

        iou_b1 = intersection_over_union(predictions[..., 21:25], target[..., 21:25])
        iou_b2 = intersection_over_union(predictions[..., 26:30], target[..., 21:25])

        # 두 박스 중 IOU가 더 큰 녀석을 찾습니다. (ious: 값, bestbox: 인덱스 0 or 1)
        ious = torch.cat([iou_b1.unsqueeze(0), iou_b2.unsqueeze(0)], dim=0)
        iou_maxes, bestbox = torch.max(ious, dim=0)

        # 객체가 있는 셀인지 확인 (Target의 confidence가 1인 곳)
        exists_box = target[..., 20].unsqueeze(3) # (Batch, 7, 7, 1)

        # ---------------------------------------------------------
        # 2. Box Coordinates Loss (위치 에러)
        # ---------------------------------------------------------
        # 예측값 중 '책임 있는 박스'의 좌표만 가져옵니다.
        box_predictions = exists_box * (
            (
                bestbox * predictions[..., 26:30] # Box 2가 베스트면 Box 2 선택
                + (1 - bestbox) * predictions[..., 21:25] # Box 1이 베스트면 Box 1 선택
            )
        )
        box_targets = exists_box * target[..., 21:25]

        # 너비/높이에 루트 씌우기 (음수 방지를 위해 절대값 후 sign 복구)
        # 논문: 작은 박스의 오차를 더 크게 반영하기 위함
        box_predictions[..., 2:4] = torch.sign(box_predictions[..., 2:4]) * torch.sqrt(
            torch.abs(box_predictions[..., 2:4] + 1e-6)
        )
        box_targets[..., 2:4] = torch.sqrt(box_targets[..., 2:4])

        # (Batch, 7, 7, 4) -> (Batch * 7 * 7, 4)로 펴서 MSE 계산
        box_loss = self.mse(
            torch.flatten(box_predictions, end_dim=-2),
            torch.flatten(box_targets, end_dim=-2),
        )

        # ---------------------------------------------------------
        # 3. Object Loss (객체가 있는 경우의 신뢰도 에러)
        # ---------------------------------------------------------
        # 책임 있는 박스의 confidence만 가져옵니다.
        pred_box = (
            bestbox * predictions[..., 25:26] + (1 - bestbox) * predictions[..., 20:21]
        )

        # 정답은 1 (객체가 있으니까)
        object_loss = self.mse(
            torch.flatten(exists_box * pred_box),
            torch.flatten(exists_box * target[..., 20:21]),
        )

        # ---------------------------------------------------------
        # 4. No Object Loss (객체가 없는 경우의 신뢰도 에러)
        # ---------------------------------------------------------
        # 여기서는 두 박스 모두 '0'을 예측해야 합니다.

        # (1) 원래 객체가 없는 셀의 Box 1, Box 2
        # (2) 객체가 있지만 책임지지 않는(IOU가 낮은) 박스

        no_object_loss = self.mse(
            torch.flatten((1 - exists_box) * predictions[..., 20:21], start_dim=1),
            torch.flatten((1 - exists_box) * target[..., 20:21], start_dim=1),
        )
        no_object_loss += self.mse(
            torch.flatten((1 - exists_box) * predictions[..., 25:26], start_dim=1),
            torch.flatten((1 - exists_box) * target[..., 20:21], start_dim=1),
        )

        # (팁) 객체가 있는 셀의 '진 박스(loser box)'도 no_obj loss에 포함시켜야 하지만
        # 코드가 너무 복잡해져서 간단한 구현에서는 생략하기도 합니다.
        # 하지만 정석은 포함시키는 것입니다. (여기선 일단 생략)

        # ---------------------------------------------------------
        # 5. Class Loss (클래스 확률 에러)
        # ---------------------------------------------------------
        class_loss = self.mse(
            torch.flatten(exists_box * predictions[..., :20], end_dim=-2),
            torch.flatten(exists_box * target[..., :20], end_dim=-2),
        )

        # ---------------------------------------------------------
        # 6. 최종 Loss 합산
        # ---------------------------------------------------------
        loss = (
            self.lambda_coord * box_loss  # 좌표 에러 가중치 5
            + object_loss
            + self.lambda_noobj * no_object_loss # 배경 에러 가중치 0.5
            + class_loss
        )

        return loss

In [12]:
import torch
import torch.nn as nn

# -----------------------------------------------------------------------------
# 1. IOU 계산 함수 (Loss 함수 내부에서 사용)
# -----------------------------------------------------------------------------
def intersection_over_union(boxes_preds, boxes_labels, box_format="midpoint"):
    """
    boxes_preds shape: (BATCH_SIZE, 7, 7, 4)
    boxes_labels shape: (BATCH_SIZE, 7, 7, 4)
    """

    # 입력 박스가 (x, y, w, h) 형태라고 가정 (midpoint format)
    # x, y는 셀의 중심, w, h는 너비와 높이

    box1_x = boxes_preds[..., 0:1]
    box1_y = boxes_preds[..., 1:2]
    box1_w = boxes_preds[..., 2:3]
    box1_h = boxes_preds[..., 3:4]

    box2_x = boxes_labels[..., 0:1]
    box2_y = boxes_labels[..., 1:2]
    box2_w = boxes_labels[..., 2:3]
    box2_h = boxes_labels[..., 3:4]

    # (x,y,w,h) -> (x1,y1,x2,y2) 모서리 좌표로 변환
    box1_x1 = box1_x - box1_w / 2
    box1_y1 = box1_y - box1_h / 2
    box1_x2 = box1_x + box1_w / 2
    box1_y2 = box1_y + box1_h / 2

    box2_x1 = box2_x - box2_w / 2
    box2_y1 = box2_y - box2_h / 2
    box2_x2 = box2_x + box2_w / 2
    box2_y2 = box2_y + box2_h / 2

    # 교집합(Intersection) 영역 계산
    x1 = torch.max(box1_x1, box2_x1)
    y1 = torch.max(box1_y1, box2_y1)
    x2 = torch.min(box1_x2, box2_x2)
    y2 = torch.min(box1_y2, box2_y2)

    # .clamp(0)은 음수를 0으로 만듦 (교집합이 없는 경우)
    intersection = (x2 - x1).clamp(0) * (y2 - y1).clamp(0)

    box1_area = abs((box1_x2 - box1_x1) * (box1_y2 - box1_y1))
    box2_area = abs((box2_x2 - box2_x1) * (box2_y2 - box2_y1))

    # 합집합(Union) = A + B - 교집합
    union = box1_area + box2_area - intersection + 1e-6 # 0 나누기 방지

    return intersection / union

# -----------------------------------------------------------------------------
# 2. YoloLoss 클래스 (완전 구현)
# -----------------------------------------------------------------------------
class YoloLoss(nn.Module):
    def __init__(self, S=7, B=2, C=20):
        super(YoloLoss, self).__init__()
        self.mse = nn.MSELoss(reduction="sum")
        self.S = S
        self.B = B
        self.C = C
        self.lambda_noobj = 0.5
        self.lambda_coord = 5

    def forward(self, predictions, target):
        # predictions shape: (BATCH, 7*7*30) -> (BATCH, 7, 7, 30)
        predictions = predictions.reshape(-1, self.S, self.S, self.C + self.B * 5)

        # 예측값 구조 분해 (0~19: Class, 20: Conf1, 21~24: Box1, 25: Conf2, 26~29: Box2)
        # IOU 계산: 예측한 두 박스와 정답 박스 간의 IOU 측정
        iou_b1 = intersection_over_union(predictions[..., 21:25], target[..., 21:25])
        iou_b2 = intersection_over_union(predictions[..., 26:30], target[..., 21:25])

        # 두 박스 중 IOU가 높은 것을 선택 (bestbox)
        ious = torch.cat([iou_b1.unsqueeze(0), iou_b2.unsqueeze(0)], dim=0)
        iou_maxes, bestbox = torch.max(ious, dim=0) # bestbox: 0 또는 1

        # 객체가 존재하는 셀인지 확인 (Target의 confidence가 1인 곳)
        exists_box = target[..., 20].unsqueeze(3) # (BATCH, 7, 7, 1)

        # ========================
        # 1. Box Coordinates Loss
        # ========================
        # 책임 있는 박스(bestbox)의 좌표만 가져옴
        box_predictions = exists_box * (
            (bestbox * predictions[..., 26:30] + (1 - bestbox) * predictions[..., 21:25])
        )
        box_targets = exists_box * target[..., 21:25]

        # 제곱근 처리 (음수 방지를 위한 절대값 및 sign 복구)
        box_predictions[..., 2:4] = torch.sign(box_predictions[..., 2:4]) * torch.sqrt(
            torch.abs(box_predictions[..., 2:4] + 1e-6)
        )
        box_targets[..., 2:4] = torch.sqrt(box_targets[..., 2:4])

        box_loss = self.mse(
            torch.flatten(box_predictions, end_dim=-2),
            torch.flatten(box_targets, end_dim=-2),
        )

        # ========================
        # 2. Object Loss (Confidence)
        # ========================
        # 책임 있는 박스의 신뢰도
        pred_box = (
            bestbox * predictions[..., 25:26] + (1 - bestbox) * predictions[..., 20:21]
        )

        # 정답 신뢰도는 1 (Target의 20번째 인덱스)
        object_loss = self.mse(
            torch.flatten(exists_box * pred_box),
            torch.flatten(exists_box * target[..., 20:21]),
        )

        # ========================
        # 3. No Object Loss
        # ========================
        # 객체가 없는 셀의 신뢰도는 0이어야 함
        # Box 1에 대한 페널티
        no_object_loss = self.mse(
            torch.flatten((1 - exists_box) * predictions[..., 20:21], start_dim=1),
            torch.flatten((1 - exists_box) * target[..., 20:21], start_dim=1),
        )
        # Box 2에 대한 페널티
        no_object_loss += self.mse(
            torch.flatten((1 - exists_box) * predictions[..., 25:26], start_dim=1),
            torch.flatten((1 - exists_box) * target[..., 20:21], start_dim=1),
        )

        # ========================
        # 4. Class Loss
        # ========================
        class_loss = self.mse(
            torch.flatten(exists_box * predictions[..., :20], end_dim=-2),
            torch.flatten(exists_box * target[..., :20], end_dim=-2),
        )

        loss = (
            self.lambda_coord * box_loss
            + object_loss
            + self.lambda_noobj * no_object_loss
            + class_loss
        )

        return loss

# -----------------------------------------------------------------------------
# 3. 테스트 실행 (Sanity Check)
# -----------------------------------------------------------------------------
def test_loss():
    S = 7  # 그리드 크기
    B = 2  # 박스 개수
    C = 20 # 클래스 개수

    # Loss 함수 인스턴스 생성
    criterion = YoloLoss(S=S, B=B, C=C)

    # 가짜 데이터 생성 (Batch Size = 2)
    # Predictions: (N, 1470) - 모델의 출력은 보통 Flatten 되어 나옴
    # 1470 = 7 * 7 * (20 + 2 * 5)
    preds = torch.randn(2, 1470)

    # Target: (N, 7, 7, 25)
    # 25 = 20(클래스) + 1(신뢰도) + 4(좌표)
    # 실제 데이터셋에서는 정답 박스가 셀당 1개라고 가정하므로 채널이 25개입니다.
    target = torch.randn(2, S, S, C + 5)

    # Loss 계산
    loss = criterion(preds, target)

    print(f"Loss Value: {loss.item()}")
    print("Sanity Check 성공! 에러 없이 Loss 값이 계산되었습니다.")

if __name__ == "__main__":
    test_loss()

Loss Value: nan
Sanity Check 성공! 에러 없이 Loss 값이 계산되었습니다.


In [13]:
def test_loss():
    S = 7
    B = 2
    C = 20

    criterion = YoloLoss(S=S, B=B, C=C)

    # 예측값은 그대로 둡니다 (모델 출력은 음수가 나올 수 있고, 코드 내부에서 abs 처리함)
    preds = torch.randn(2, 1470)

    # [수정] Target 데이터는 무조건 0~1 사이의 양수여야 합니다 (너비, 높이 때문에)
    # torch.rand는 0과 1 사이의 균등 분포를 생성하므로 음수가 안 나옵니다.
    target = torch.rand(2, S, S, C + 5)

    loss = criterion(preds, target)

    print(f"Loss Value: {loss.item()}")

    if not torch.isnan(loss):
        print("Sanity Check 완벽 성공! nan 없이 숫자가 잘 나왔습니다.")
    else:
        print("여전히 nan입니다. 코드를 다시 확인해야 합니다.")

if __name__ == "__main__":
    test_loss()

Loss Value: 1886.8001708984375
Sanity Check 완벽 성공! nan 없이 숫자가 잘 나왔습니다.
